# SHAKTI FEniCS code

The code requires [FEniCSx](https://fenicsproject.org). 

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, time
sys.path.insert(0, '../source')

Set model options in main.py and then run the code: 

In [ ]:
t1 = time.time()
!mpirun -np 4 python3 ../source/main.py setup_cooke2
t2 = time.time()
print(f'total simulation time: {t2-t1} s')